In [ ]:
from scipy.ndimage import zoom
import numpy as np

In [ ]:
a = np.ndarray([973, 1057], dtype=np.float32)

In [ ]:
for ii in range(4):
    print(ii, np.array(a.shape)*0.5**ii)
    print(ii, zoom(a, 0.5**ii).shape)

In [ ]:
n1 = set([1, 2, 3, 4])
n2 = set([2, 4, 6])
print(sorted(list(n1 & n2)))

In [ ]:
if sorted(list(n1 & n2)):
     print(1)
else:
     print(0)
        

In [ ]:
2*np.pi*9.87903331E-06*1000/(12.4/18)/100

In [ ]:
1e13/40000/40000

In [ ]:
1e13*15/(3.7e13*20)

In [1]:
%matplotlib qt
import matplotlib.pyplot as plt
import h5py
from scipy.ndimage import fourier_shift, zoom, median_filter as mf, gaussian_filter as gf
from TXM_Sandbox.TXM_Sandbox.utils.xanes_math import (mrtv_reg1, mrtv_reg2, mrtv_reg3,
                                                      mrtv_mpc_combo_reg, tv_l1_pixel, 
                                                      tv_l1_subpixel, mrtv_ls_combo_reg)
import numpy as np
from skimage.measure import shannon_entropy
from timeit import timeit
import time

In [2]:
import imagej
ij = imagej.init('/home/xiao/software/Fiji.app', headless=False)
ijui = ij.ui()
ijui.showUI()
from jnius import autoclass
wm = autoclass('ij.WindowManager')

Added 491 JARs to the Java classpath.


In [7]:
#fn = '/run/media/xiao/Data/data/2D_xanes/multipos_2D_xanes_scan2_id_23669_repeat_00_pos_01.h5'
#fn = '/run/media/xiao/Data/data/2D_xanes/multipos_2D_xanes_scan2_id_61201_repeat_00_pos_01.h5'
#fn = '/run/media/xiao/Data/data/2D_xanes/xanes_scan2_id_54740.h5'
#fn = '/run/media/xiao/Data/data/Weijiang_2020Q1/xanes_scan2_id_{}.h5'
fn = '/run/media/xiao/Data/data/Weijiang_2020Q1/data/multipos_2D_xanes_scan2_id_{0}_repeat_{1}_pos_{2}.h5'
idx = 61201
rep = '00'
pos = '00'
pair = [50, 0]
#pair = [43, 50]
#roi = [472, 1563, 1030, 2035]
#roi = [503, 1963, 603, 1679]

roi = [0, 1080, 0, 1280]
#roi = [392, 392+535, 110, 708]
with h5py.File(fn.format(idx, rep, pos), 'r') as f:
    fixed = ((f['/img_xanes'][pair[0], roi[0]:roi[1], roi[2]:roi[3]] - f['/img_dark'][0, roi[0]:roi[1], roi[2]:roi[3]])/
             (f['/img_bkg'][pair[0], roi[0]:roi[1], roi[2]:roi[3]] - f['/img_dark'][0, roi[0]:roi[1], roi[2]:roi[3]]))
    moving = ((f['/img_xanes'][pair[1], roi[0]:roi[1], roi[2]:roi[3]] - f['/img_dark'][0, roi[0]:roi[1], roi[2]:roi[3]])/
              (f['/img_bkg'][pair[1], roi[0]:roi[1], roi[2]:roi[3]] - f['/img_dark'][0, roi[0]:roi[1], roi[2]:roi[3]]))
    
fixed[np.isnan(fixed)] = 1
fixed[np.isinf(fixed)] = 1
moving[np.isnan(moving)] = 1
moving[np.isinf(moving)] = 1
print(fixed.shape, moving.shape)

(1080, 1280) (1080, 1280)


In [8]:
f = -np.log(fixed)
m = -np.log(moving)

In [9]:
tv, tv_id, shift, s = mrtv_ls_combo_reg(ls_w=100, levs=2, wz=10, sp_wz=6, sp_step=0.5, imgs=[f, m])
print(f'shift:{shift}', '\n', f's:{s}')

Tue Nov  3 19:22:05 2020
Tue Nov  3 19:22:05 2020
shift:[[-12.    8. ]
 [  1.   -3. ]
 [  0.    0.5]] 
 s:[-11.    5.5]


In [ ]:
tv, tv_id, shift, s = mrtv_reg3(7, 10, 8, 0.5, [f, m])
print(f'shift:{shift}', '\n', f's:{s}')

In [11]:
diff_img1 = f - np.real(np.fft.ifftn(fourier_shift(np.fft.fftn(m), [s[0], s[1]])))
diff_img2 = gf(f, 3) - gf(np.real(np.fft.ifftn(fourier_shift(np.fft.fftn(m), [s[0], s[1]]))), 3)
ijui.show(ij.py.to_java(f))
wm.getCurrentImage().setTitle('fixed')
ijui.show(ij.py.to_java(np.real(np.fft.ifftn(fourier_shift(np.fft.fftn(m), [s[0], s[1]])))))
wm.getCurrentImage().setTitle('translated moving')
ijui.show(ij.py.to_java(m))
wm.getCurrentImage().setTitle('moving')
ijui.show(ij.py.to_java(diff_img1))
wm.getCurrentImage().setTitle('fixed - moving')
ijui.show(ij.py.to_java(np.log(np.abs(np.diff(diff_img2, axis=0, prepend=1))+np.abs(np.diff(diff_img2, axis=1, prepend=1)))))
wm.getCurrentImage().setTitle('derivative_differential')

In [ ]:
mk = np.ones(f.shape)

tv_v = []
for ii in range(100):
    tv_v.append(tv_l1_pixel(f, m, mk, [2*(ii-50), 0]))

tv_h = []
for ii in range(100):
    tv_h.append(tv_l1_pixel(f, m, mk, [0, 2*(ii-50)]))
    

In [ ]:
plt.figure(1)
plt.plot(tv_h)

plt.figure(2)
plt.plot(tv_v)

print(np.array(tv_h).argmin(), np.array(tv_v).argmin())

In [ ]:
fz = zoom(f, 0.5**(len(shift)-2))
mz = zoom(m, 0.5**(len(shift)-2))

ijui.show(ij.py.to_java(fz))
wm.getCurrentImage().setTitle('fz')
ijui.show(ij.py.to_java(mz))
wm.getCurrentImage().setTitle('mz')
ijui.show(ij.py.to_java(diff_img1))
wm.getCurrentImage().setTitle('fz - mz')

In [ ]:
#tvm1, tvi1, s1, ss1 = mrtv_reg1(fixed, moving, levs=5, wz=10, sp_wz=10, sp_step=0.5)
tvm2, tvi2, s2, ss2 = mrtv_reg2(fixed, moving, levs=6, wz=10, sp_wz=10, sp_step=0.5)

In [ ]:
t0 = time.perf_counter()
print('start at : 0sec')
f = zoom(fixed, 0.0625)
m = zoom(moving, 0.0625)
mk = np.ones(f.shape)
for ii in range(100):
    #zoom(fixed, 0.125)
    tv_l1_pixel(f, m, mk, [1, 1])
    #tv_l1_subpixel(f, m, mk, [1, 1])
    #mrtv_reg2(f, m, levs=2, wz=6, sp_wz=6, sp_step=0.5)
print('level 0 finishes at : {}sec'.format(time.perf_counter() - t0))

f = zoom(fixed, 0.125)
m = zoom(moving, 0.125)
mk = np.ones(f.shape)
for ii in range(36):
    tv_l1_pixel(f, m, mk, [1, 1])
print('level 1 finishes at : {}sec'.format(time.perf_counter() - t0))

f = zoom(fixed, 0.25)
m = zoom(moving, 0.25)
mk = np.ones(f.shape)
for ii in range(36):
    tv_l1_pixel(f, m, mk, [1, 1])
print('level 2 finishes at : {}sec'.format(time.perf_counter() - t0))

f = zoom(fixed, 0.5)
m = zoom(moving, 0.5)
mk = np.ones(f.shape)
for ii in range(36):
    tv_l1_pixel(f, m, mk, [1, 1])
print('level 3 finishes at : {}sec'.format(time.perf_counter() - t0))

f = fixed
m = moving
mk = np.ones(f.shape)
for ii in range(36):
    tv_l1_pixel(f, m, mk, [1, 1])
print('level 4 finishes at : {}sec'.format(time.perf_counter() - t0))

for ii in range(36):
    tv_l1_subpixel(f, m, mk, [1, 1])
print('level 5 finishes at : {}sec'.format(time.perf_counter() - t0))

mrtv_reg2(f, m, levs=6, wz=6, sp_wz=6, sp_step=0.5)
print('mrtv finishes at : {}sec'.format(time.perf_counter() - t0))

In [ ]:
print(len(tv_id))
print(tv[5].flatten()[tv_id[-1]])

In [ ]:
plt.figure(10)
plt.imshow(fixed)

In [ ]:
help(time.perf_counter)
time.perf_counter()

In [ ]:
fig, ax = plt.subplots(1, len(tvm2))
for ii in tvm2.keys():
    ax[ii].plot(tvm2[ii].flatten())

In [ ]:
fig, ax = plt.subplots(1, len(tv))
for ii in tv.keys():
    ax[ii].imshow(tv[ii])

In [ ]:
#diff_img1 = fixed - np.real(np.fft.ifftn(fourier_shift(np.fft.fftn(moving), [ss1[0], ss1[1]])))
#plt.figure(3)
#plt.imshow(diff_img1)

diff_img1 = fixed - moving
plt.figure(3)
plt.imshow(diff_img1)
print(ss2)
diff_img2 = fixed - np.real(np.fft.ifftn(fourier_shift(np.fft.fftn(moving), [ss2[0], ss2[1]])))
plt.figure(4)
plt.imshow(diff_img2)

In [ ]:
np.array([1091, 1005])*(0.5**(levs-2))

In [ ]:
ii = 2
w = sch_config[ii]['wz']
step = sch_config[ii]['step']
f = zoom(fixed, step)
m = zoom(moving, step)
mk = np.zeros(m.shape, dtype=np.int8)
# mk[mk.shape[0]//4:-mk.shape[0]//4, mk.shape[1]//4:-mk.shape[1]//4] = 1
mk[int(wz/2):-int(wz/2), int(wz/2):-int(wz/2)] = 1

shift = np.array([[0, -1], [0, -1], [0, 1]])
s= [0, 0]
#s= np.array([0, 0])
for kk in range(ii):
    s += shift[kk]*(2**(ii-kk))
print(s)

In [ ]:
print(f.shape, m.shape, step)
print(s)
plt.figure(1)
plt.imshow(mk)
plt.figure(2)
plt.imshow(m)
plt.figure(3)
plt.imshow(f)

In [ ]:
tv = []
for jj in range(w**2):
    shift = [int((jj//w-int(w/2))+s[0]), int((jj%w-int(w/2))+s[1])]
    tv.append(tv_l1_pixel(f, m, mk, shift))

In [ ]:
plt.figure(10)
plt.plot(tv)

In [ ]:
idx = np.array(tv).argmin()
print(idx, w)

In [ ]:
#idx = 55
ss3 = np.array([(idx//w-int(w/2)), (idx%w-int(w/2))])
print(ss3)
#print(len(tv))
#print(tv[39:41])76543
#print(ss3[0], ss3[1])
#print(int(w/2))

In [ ]:
print(tv[54:60])

In [ ]:
tem = s + ss3 
print(tem)
diff_img3 = f - np.real(np.fft.ifftn(fourier_shift(np.fft.fftn(m), tem)))
plt.figure(5)
plt.imshow(diff_img3)

In [ ]:
print('original image size'.center(50, '-'))
print(fixed.shape)
print('hierarchical image size'.center(50, '-'))
for i in range(5):
    print(i, ':', sch_config[i]['wz'], sch_config[i]['step'], np.int32(np.array(fixed.shape)*sch_config[i]['step']))

In [ ]:
levs = 6
wz=10
sp_wz=10; sp_step=0.5
sch_config = {}
sch_config[levs-1] = {'wz':sp_wz, 'step':sp_step}
for ii in range(levs-1):
    sch_config[levs-2-ii] = {'wz':wz, 'step':0.5**ii}
print(sch_config)

In [ ]:
fig, ax = plt.subplots(1, len(tvm2), num=100)
for ii in tvm2.keys():
    ax[ii].plot(tvm2[ii].flatten())

In [ ]:
print('sep'.center(50, '-'))
print(tvi2)
print(s2)
print(ss2)

In [ ]:
print('line "s2"'.center(50, '-'))
print(s2)
ii = 5
w = sch_config[ii]['wz']
step = sch_config[ii]['step']

s = np.array([0, 0])
if ii == (s2.shape[0]-1):
    sp_step = 0.5
    f = fixed
    m = moving
    mk = np.zeros(m.shape, dtype=np.int8)
    mk[int(wz/2):-int(wz/2), int(wz/2):-int(wz/2)] = 1
    print('line f.shape'.center(50, '-'))
    print(f'level {ii}:' ,f.shape)
    print('line by line "s"'.center(50, '-'))
    print('route 0')
    for kk in range(s2.shape[0]-1):
        s = s + s2[kk]*(2**(ii-1-kk))
        print(kk, s)
else:
    sp_step = 1
    f = zoom(fixed, step)
    m = zoom(moving, step)
    mk = np.zeros(m.shape, dtype=np.int8)
    mk[int(wz/2):-int(wz/2), int(wz/2):-int(wz/2)] = 1
    print('line f.shape'.center(50, '-'))
    print(f'level {ii}:' ,f.shape)
    print('line by line "s"'.center(50, '-'))
    print('route 1')
    for kk in range(ii):
        s = s + s2[kk]*(2**(ii-kk-1))
        print(kk, s)
s = np.int32(s)

adj = [0, 0]
diff_img3 = f - np.real(np.fft.ifftn(fourier_shift(np.fft.fftn(m), s+adj)))
plt.figure('last level')
plt.imshow(diff_img3)
print('line: ideal translation'.center(50, '-'))
print(ii, 's:', s, 'adjusted s:', s+adj)

print('line tv'.center(50, '-'))

tv2 = []
for jj in range(40, 50):
    ss3 = np.array([(jj//w-int(w/2)), (jj%w-int(w/2))])*sp_step
    tem = s + ss3
    
    #diff_img3 = mf(f - np.real(np.fft.ifftn(fourier_shift(np.fft.fftn(m), tem))), 3)
    #tv_img3 = mf(((np.abs(np.diff(diff_img3, axis=0, prepend=1))+np.abs(np.diff(diff_img3, axis=1, prepend=1)))*mk), 3)
    
    diff_img3 = gf(f, 5) - gf(np.real(np.fft.ifftn(fourier_shift(np.fft.fftn(m), tem))), 5)
    #diff_img3 /= np.sqrt((diff_img3**2).sum())
    tv_img3 = ((np.abs(np.diff(diff_img3, axis=0, prepend=1))+np.abs(np.diff(diff_img3, axis=1, prepend=1)))*mk)
    #tv_img3 /= np.sqrt((tv_img3**2).sum())
    
    
    fig, ax = plt.subplots(1, 2, num=jj)
    ax[0].imshow(diff_img3)
    ax[1].imshow(tv_img3)
    fig.suptitle(str(tem))
    

    tv2.append(tv_img3.std())
    print(jj, 'tem:', tem, 'ss3:', ss3, 'tv std:', round(tv_img3.std(), 10), 'tv sum:', round(tv_img3.sum(), 10))

In [ ]:
plt.figure(1000)
plt.plot(tv2)

In [ ]:
plt.figure(10000)
plt.imshow(mf(tv_img3, 3))

In [ ]:
diff_img3 = f - np.roll(m, [0, 0], axis=[0, 1])
plt.figure(101)
plt.imshow(np.abs(np.diff(diff_img3, axis=0, prepend=1)))
plt.figure(102)
plt.imshow(np.abs(np.diff(diff_img3, axis=1, prepend=1)))

In [ ]:
diff_img4 = f - np.roll(m, [0, -3], axis=[0, 1])
plt.figure(103)
plt.imshow(np.abs(np.diff(diff_img4, axis=0, prepend=1)))
plt.figure(104)
plt.imshow(np.abs(np.diff(diff_img4, axis=1, prepend=1)))

In [ ]:
diff_img3 = f - np.roll(m, [0, 0], axis=[0, 1])
ijui.show(ij.py.to_java(np.abs(np.diff(mf(diff_img3, 3), axis=0, prepend=1))))
wm.getCurrentImage().setTitle('diff_img3_axis0')
ijui.show(ij.py.to_java(np.abs(np.diff(mf(diff_img3, 3), axis=1, prepend=1))))
wm.getCurrentImage().setTitle('diff_img3_axis1')

diff_img4 = f - np.roll(m, [0, -3], axis=[0, 1])
ijui.show(ij.py.to_java(np.abs(np.diff(mf(diff_img4, 3), axis=0, prepend=1))))
wm.getCurrentImage().setTitle('diff_img4_axis0')
ijui.show(ij.py.to_java(np.abs(np.diff(mf(diff_img4, 3), axis=1, prepend=1))))
wm.getCurrentImage().setTitle('diff_img4_axis1')

In [ ]:
ijui.show(ij.py.to_java(np.diff(diff_img3, axis=0, prepend=1)))
wm.getCurrentImage().setTitle('raw_diff_img3_axis0')
ijui.show(ij.py.to_java(np.diff(diff_img3, axis=1, prepend=1)))
wm.getCurrentImage().setTitle('raw_diff_img3_axis1')

ijui.show(ij.py.to_java(np.diff(diff_img4, axis=0, prepend=1)))
wm.getCurrentImage().setTitle('raw_diff_img4_axis0')
ijui.show(ij.py.to_java(np.diff(diff_img4, axis=1, prepend=1)))
wm.getCurrentImage().setTitle('raw_diff_img4_axis1')

In [ ]:
1e7*1e7*5e6/8/8/8

In [ ]:
5e6/20*7/24/265

In [ ]:
1e4*1e4*5e3/100/30/20*7/24/365

In [ ]:
0.35*1800*4

In [ ]:
0.1*1800*4

In [ ]:
1e4*1e4*5e3/60/60/60*2/3/24/365

In [ ]:
10*10*5*3/60/24